# Imports

Packages (pip freeze) in requirements.txt


In [1]:
%load_ext autoreload
%autoreload 2

# import helper functions to generate graphs and timeseries
from helper_functions import *

# keras
from keras.models import *
from keras.layers import *

# z normalization
from sklearn.preprocessing import StandardScaler

# metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# misc
from tqdm.notebook import trange
import matplotlib.pyplot as plt


# Training Function

## FFNN (Our approach)

In [2]:
def train_model_fnn(X_train,y_train, n_features):
    cb = EarlyStopping(monitor='val_loss', mode='min',patience=7)

    #CNN architecture
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(n_features,)))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(200, activation='tanh'))
    model.add(Dense(100, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    
    #Save the rmsesparse_categorical_crossentropy
    history = model.fit(X_train, y_train, epochs=200, validation_split=0.1, verbose=0, callbacks=[cb])
    return model
 


## CNN [S. Machado et al.]

In [3]:
def train_model_cnn(X_train,y_train, n_features):
    """
        Function to train a cnn model
            Model architecture follows the structure used in [S. Machado et al.]
    """
    cb = EarlyStopping(monitor='val_loss', mode='min',patience=7)

    #CNN architecture
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, strides=2, activation='relu', input_shape=(n_features,1)))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
    model.add(GlobalMaxPooling1D())

    model.add(Dropout(0.2))
    model.add(Dense(200, activation='tanh'))
    model.add(Dense(100, activation='tanh'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse')
    
    # train
    history = model.fit(X_train, y_train, epochs=200, validation_split=0.1, verbose=0, callbacks=[cb])
    return model

## Accuracy and Identifiability Gap

In [4]:
def get_acc_idgap(pred, y, dis_mpred, con_mpred):
    """
        Calculate the accuracy of the model and the identifiability gap

        accuracy:               measures the number of correct predictions
        identifiability gap:    distance between the lowest predicted connected pair and 
                                the highest predicted disconnected pair
                            
        input:
            pred:   predictions 
            y:      target
            dis_mpred: disconnected preds
            con_mpred: connected preds

        output:
            (accuracy, identifiability gap)

    """
    # - - - testing 
    #Divide the connected and disconnected pairs values
    test_size = len(y)
    idd = y < 2
    idd = np.squeeze(idd)

    idc = y > 1
    idc = np.squeeze(idc)

    nc = np.sum(idc)

    # predict and Normalize the values
    true = y - 1

    #Initialize the structures to save the data
    con = np.zeros((nc))
    dis = np.zeros((test_size-nc))

    #Split the values belonging to connected and disconnected pairs
    c2,c3 = 0,0
    for i in range(len(true)):
        if(true[i]>0):
            con[c2] = pred[i]
            c2+=1
        else:
            dis[c3] = pred[i]
            c3+=1

    #Threshold
    mint = np.max(dis)
    maxt = np.min(con)

    #Compute metrics
    idgap = maxt - ((mint+maxt)/2)

    #Predicts the weight of the connection and classifies the data
    truec_nn = np.sum(dis_mpred<1.5)
    trued_nn = np.sum(con_mpred>1.5)

    tall = truec_nn + trued_nn

    acc = tall/(len(dis_mpred)+len(con_mpred))*100


    #If the id gap is negative then we cant correctly classify the pairs as disconnected or connected
    if(idgap < 0):
        idgap=0
    
    return acc, idgap

# Training our model

## Generate test set

In [5]:
# graph parameters
undirected = True
p = 0.5  
sz = 40
ss = 0
se = 20

# timeseries parameters
tsize = 100000
x0 = 0

# noise values
beta = 2
alpha = 1
c = 0.9
rho = 0.75

# get adjacency (adj) and interaction (A) matrices
adj = get_adjacency(sz,p,undirected)
A = get_A(adj,c,rho)

# generate noise
noise = generate_noise(sz, tsize, alpha, beta)

# generate timeseries
x = generate_timeseries(A,tsize,x0,noise)

# partial observability
x = x[:, ss:se]
A = A[ss:se, ss:se]
sz = se - ss

# generate features
features_scaled = get_inverted_features(x, 100)
scaler = StandardScaler()
X_test = scaler.fit_transform(features_scaled)
y_test = get_target(A)



## Training parameters

In [6]:
# graph parameters
undirected = True

# define the range of noise variance
alpha = 1
x0 = 0            
c = 0.9
rho = 0.75

# training parameters
n_datasets = 20
n_runs = 10         # pick the best performing model on the test set over 10 models 


# training is done with multiple datasets with varying correlated noise level (beta), 
# number of samples (tsize) and number of nodes (sz) 
betas = np.linspace(0,50,n_datasets).astype(int)
offset = 20000
tssizes = (betas+1)*offset
print(betas)
print(tssizes)

n_datasets = tssizes.shape[0]

# save models to pick best
models = []


[ 0  2  5  7 10 13 15 18 21 23 26 28 31 34 36 39 42 44 47 50]
[  20000   60000  120000  160000  220000  280000  320000  380000  440000
  480000  540000  580000  640000  700000  740000  800000  860000  900000
  960000 1020000]


## Run

In [7]:
# iterate over number of runs
for run in (t := trange(n_runs)):

    # concatenate datasets for training 
    for i in range(n_datasets):

        # random graph size (number of nodes)
        sz = np.random.randint(30, 80) 

        # generate the adjacency and A matrices
        adj = get_adjacency(sz,p,undirected)
        A = get_A(adj,c,rho)
        y = get_target(A)

        # generate noise 
        noise = generate_noise(sz, tssizes[i], alpha, betas[i])

        # generates the synthetic time series
        x = generate_timeseries(A,tssizes[i],x0,noise)

        # features
        features_scaled = get_inverted_features(x, 100)
        scaler = StandardScaler()
        features_ss = scaler.fit_transform(features_scaled)

        if i == 0:
            ys = y
            features_ss2 = features_ss

        else:
            ys = np.concatenate((ys,y),axis=0)
            features_ss2 = np.concatenate((features_ss2, features_ss), axis=0)

    # shuffle datasets
    n_samples = features_ss2.shape[0]
    perm = np.random.permutation(n_samples)
    features_ss2 = features_ss2[perm,:]
    ys = ys[perm,:]

    # train
    model = train_model_fnn(features_ss2, ys, features_ss2.shape[1], log_name=f'_s_{run}')

    # test
    idd = y_test < 2
    idd = np.squeeze(idd)

    idc = y_test > 1
    idc = np.squeeze(idc)

    # calculate accuracy and idgap on test set
    pred = model.predict(X_test, verbose=0)
    dis_mpred = model.predict(X_test[idd,:], verbose=0)
    con_mpred = model.predict(X_test[idc,:], verbose=0)
    acc, idgap = get_acc_idgap(pred, y_test, dis_mpred, con_mpred)

    # save model and performance
    models.append((model, acc, idgap))

    t.set_description(f'acc {acc} idgap {idgap}')



  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Saving best model

In [ ]:
def sort_models(models):
    """
        sort models by accuracy
        in tiebreak:
            sort models by the maximum of idgap
    """
    return sorted(models, key=lambda x: ( -x[1], abs(0.5 - x[2]) ) )

file_name = "model_ffnn"

# sort models based on accuracy and idgap
models = sort_models(models)
_model = models[0]
model = _model[0]
print(_model)
print(models)

save_model(model,file_name+'_ss')